In [2]:
from sklearn import datasets
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  
import numpy as np
import pandas as pd
import sys

In [3]:
# reading data 
Location = r'C:\Users\inkpathak\Desktop\KP way Text Basic\Text Basic\server text.csv'
server_data = pd.read_csv(Location)

In [4]:
server_data.head()

,Date,Notes,Notes_clean,Prediction
0,04/02/2018 10:13:07,Use ADRCI or Support Workbench to package the ...,use adrci support workbench package incident s...,Dumping diagnostic
1,03/02/2018 06:13:34,Current log# 11 seq# 31124 mem# 0: +REDO_DG04/...,current merlp tp onlinelog group current merlp...,LGWR switch
2,03/02/2018 06:13:35,Archived Log entry 182454 added for thread 3 s...,archived log entry added thread sequence id dest,Archived Log entry
3,04/02/2018 20:13:06,Running KSFV I/O slave I601 os pid=24411,running ksfv slave os,KSFV
4,01/02/2018 16:42:52,TNS-00505: Operation timed out.01/02/2018 16:4...,tns operation timed nt os err code client addr...,Fatal NI


What is tokenizing ?

Splitting a document based on it's word component is called tokenizing'

How to convert tokenized bag of words into numbers 
    * Count vectorizer or to create Document Term Matrix  for each of the document how many timne each word has appeared ?
    * TFIDF ---- will discuss in some time    

In [7]:
server_data['Notes_clean'][1]

'current merlp tp onlinelog group current merlp tp onlinelog group thread advanced log sequence lgwr switch'

In [8]:
server_data['Notes_clean'][0]
                    

'use adrci support workbench package incident see note oracle support error packaging details dumping diagnostic data requested sweep completed'

In [9]:
server_data['Notes_clean'][5]

'errors file dborafiles admin diag rdbms merlp tp trace trc ora error auto execute job lu lu j default consumer ora queue sys lu messages q exist ora lu log util adm line ora line'

In [11]:
server_data['Notes_clean'][10]

'archived log entry added thread sequence id dest current merlp tp onlinelog group thread advanced log sequence lgwr switch current merlp tp onlinelog group'

### TFIDF example:
    

In [12]:
bowA=server_data['Notes_clean'][5]
bowB=server_data['Notes_clean'][10]

In [13]:
bowA=bowA.split(" ")
bowB=bowB.split(" ")

In [14]:
bowA

['errors',
 'file',
 'dborafiles',
 'admin',
 'diag',
 'rdbms',
 'merlp',
 'tp',
 'trace',
 'trc',
 'ora',
 'error',
 'auto',
 'execute',
 'job',
 'lu',
 'lu',
 'j',
 'default',
 'consumer',
 'ora',
 'queue',
 'sys',
 'lu',
 'messages',
 'q',
 'exist',
 'ora',
 'lu',
 'log',
 'util',
 'adm',
 'line',
 'ora',
 'line']

In [15]:
wordSet = set(bowA).union(set(bowB))

In [18]:
wordSet

{'added',
 'adm',
 'admin',
 'advanced',
 'archived',
 'auto',
 'consumer',
 'current',
 'dborafiles',
 'default',
 'dest',
 'diag',
 'entry',
 'error',
 'errors',
 'execute',
 'exist',
 'file',
 'group',
 'id',
 'j',
 'job',
 'lgwr',
 'line',
 'log',
 'lu',
 'merlp',
 'messages',
 'onlinelog',
 'ora',
 'q',
 'queue',
 'rdbms',
 'sequence',
 'switch',
 'sys',
 'thread',
 'tp',
 'trace',
 'trc',
 'util'}

In [19]:
# dictionary to keep the counts of the words
wordDictA=dict.fromkeys(wordSet,0)
wordDictB=dict.fromkeys(wordSet,0)

In [20]:
# creating DTM
for word in bowA:
    wordDictA[word]= 1+ wordDictA[word]
for word in bowB:
    wordDictA[word]= 1+ wordDictB[word]    

In [21]:
pd.DataFrame([wordDictA,wordDictB])

,added,adm,admin,advanced,archived,auto,consumer,current,dborafiles,default,...,queue,rdbms,sequence,switch,sys,thread,tp,trace,trc,util
0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### Why DTM is not so good idea?

    * First of all this is a discrete representation of the document and no symantics covered.
    
    * Most common words in the english langiage is "the" it consists of nearly 7% of the words we say. Next one is "of" which is nearly half of it.

#### Distribution of words in any language is a power law distribution . this forms the basis for Zip's law

#### Why TFIDF is a better choice?

TFIDF = TF(word)* IDF(word)
    Where,
    
    TF(word1) = (number of times word appears in a document) / (Total number of words in the document)
    
    IDF(word1) = log [(number of documents)/(no of documents containing word1)]
    
so TFIDF has one added advantage of IDF which acts as relative importance for each words based on 

In [24]:
def compute_TF(wordDict,bow):
    tfDict={}
    bowCount = len(bow)
    for word, count in wordDict.items():
        tfDict[word]=count/ float(bowCount)
    return tfDict    

In [25]:
tfBowA=compute_TF(wordDictA,bowA)
tfBowB=compute_TF(wordDictB,bowB)

In [26]:
import math
def compute_IDF(doclist):
    idf_Dict= {}
    N=len(docList)
    
    # lets count number of document proportion containing the selected word
    idf_Dict = dict.fromkeys(docList[0].keys(),0)
    for doc in docList:
        for word, val in doc.items():
            if val>0:
                idf_Dict[word]= idf_Dict[word] + 1
    
    for word, val in idf_Dict.items():
        idf_Dict[word]= math.log(N/float(val))
    return idf_Dict    

In [31]:
docList= [wordDictA,wordDictB]
idfs= compute_IDF(docList)

In [32]:
def compute_TFIDF(tfBow, idfs):
    tfidf={}
    for word, val in tfBow.items():
        tfidf[word]= val * idfs[word]
    return tfidf    

In [33]:
tfidfBowA= compute_TFIDF(tfBowA,idfs)
tfidfBowB= compute_TFIDF(tfBowB,idfs)

In [34]:
pd.DataFrame([tfidfBowA,tfidfBowB])

,added,adm,admin,advanced,archived,auto,consumer,current,dborafiles,default,...,queue,rdbms,sequence,switch,sys,thread,tp,trace,trc,util
0,0.019804,0.019804,0.019804,0.019804,0.019804,0.019804,0.019804,0.019804,0.019804,0.019804,...,0.019804,0.019804,0.019804,0.019804,0.019804,0.019804,0.019804,0.019804,0.019804,0.019804
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### Now we understand how TFIDF can be created , let implement the same in a Model with Latent feature calculation

In [ ]:
from sklearn.datasets import fetch_20newsgroups
categories = ['rec.sport.baseball']
dataset = fetch_20newsgroups(subset='all',shuffle=True, random_state=42, categories=categories)
corpus=dataset.data

In [36]:

from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [37]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\inkpathak\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [38]:
stopset = set(stopwords.words('english'))
stopset.update(['tp','trc',  ])

In [39]:
corpus = server_data['Notes_clean']

In [40]:
# Very important step of N-Gram as well
vectorizer = TfidfVectorizer(stop_words=stopset,use_idf=True, ngram_range=(1, 3))
X = vectorizer.fit_transform(corpus)

In [42]:
X[0]

<1x3679 sparse matrix of type '<class 'numpy.float64'>'
	with 53 stored elements in Compressed Sparse Row format>

In [44]:
X[5]

<1x3679 sparse matrix of type '<class 'numpy.float64'>'
	with 83 stored elements in Compressed Sparse Row format>

In [ ]:
print(X[0])

## Latent Symantic Analysis (Topic scoring)

input = X [matrix with "M" no of documents and "N" number of words]

Process = Perdorm SVD ( Singular Value Decomposition for this input matrix) 
        X = U * S * V-Transpose
   Where:
   
   U = M*K matrix --------------- Rows would be the document and columns will be concepts(K)
   
   S = K*K diagonal matrix ------ Elements would be the amount of variations captured from each concept(K)
   
   Vt = K*N matrix -------------- Rows will be concepts(K) and columns would be the terms---- Need to care for transpose


In [45]:
lsa = TruncatedSVD(n_components=45, n_iter=100)
lsa.fit(X)

TruncatedSVD(algorithm='randomized', n_components=45, n_iter=100,
       random_state=None, tol=0.0)

In [46]:

lsa.components_[0]

array([  1.26372733e-07,   1.26372734e-07,   1.26372734e-07, ...,
         1.49150842e-07,   1.49150842e-07,   1.49150842e-07])

In [48]:
terms = vectorizer.get_feature_names()
for i, comp in enumerate(lsa.components_): 
    termsInComp = zip (terms,comp)
    sortedTerms =  sorted(termsInComp, key=lambda x: x[1], reverse=True) [:10]
    print("Concept %d:" % i)
    for term in sortedTerms:
        print(term[0])
    print(" ")

Concept 0:
sequence
thread
added thread
added thread sequence
archived
archived log
archived log entry
entry
entry added
entry added thread
 
Concept 1:
group
onlinelog
onlinelog group
current
current onlinelog
current onlinelog group
advanced
advanced log
advanced log sequence
lgwr
 
Concept 2:
lu
ora
line
adm
adm line
adm line ora
consumer
consumer ora
consumer ora queue
default consumer
 
Concept 3:
ksfv
ksfv slave
slave
ksfv slave os
running
running ksfv
running ksfv slave
slave os
os
exiting
 
Concept 4:
sys
sys sys
sys sys sys
index
agg
dly
deleted
deleted statistics
statistics
columns
 
Concept 5:
alter system archive
archive
archive log
system archive
system archive log
alter
alter system
system
log
current merlp
 
Concept 6:
current merlp
current merlp onlinelog
merlp onlinelog
merlp onlinelog group
merlp
group current merlp
switch current merlp
group archived
group archived log
onlinelog group archived
 
Concept 7:
exiting
ksfv slave exiting
slave exiting
ksfv
ksfv slave
slav

### Next we need to see how can we extract other two matrix for our benefit